In [2]:
from diffusers import DiffusionPipeline
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

pipeline = DiffusionPipeline.from_pretrained(
    'joeySadventure/stable_diffusion_train', safety_checker=None)

scheduler = pipeline.scheduler
tokenizer = pipeline.tokenizer

del pipeline

device, scheduler, tokenizer

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/home/liyuanhang/miniconda3/envs/ldm/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


('cuda',
 PNDMScheduler {
   "_class_name": "PNDMScheduler",
   "_diffusers_version": "0.14.0",
   "beta_end": 0.012,
   "beta_schedule": "scaled_linear",
   "beta_start": 0.00085,
   "clip_sample": false,
   "num_train_timesteps": 1000,
   "prediction_type": "epsilon",
   "set_alpha_to_one": false,
   "skip_prk_steps": true,
   "steps_offset": 1,
   "trained_betas": null
 },
 CLIPTokenizer(name_or_path='/home/liyuanhang/.cache/huggingface/hub/models--joeySadventure--stable_diffusion_train/snapshots/3e8255dfdd3b0de298d7995fbee8069498d16c1d/tokenizer', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_

In [3]:
from datasets import load_dataset
import torchvision

#加载数据集
dataset = load_dataset(path='joeySadventure/dataset_pokemon', split='train')

#图像增强模块
compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
        512, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),
    torchvision.transforms.CenterCrop(512),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]),
])


def f(data):
    #应用图像增强
    pixel_values = [compose(i) for i in data['image']]

    #文字编码
    input_ids = tokenizer.batch_encode_plus(data['text'],
                                            padding='max_length',
                                            truncation=True,
                                            max_length=77).input_ids

    return {'pixel_values': pixel_values, 'input_ids': input_ids}


dataset = dataset.map(f,
                      batched=True,
                      batch_size=100,
                      num_proc=1,
                      remove_columns=['image', 'text'])

dataset.set_format(type='torch')

dataset, dataset[0]

Found cached dataset parquet (/home/liyuanhang/.cache/huggingface/datasets/joeySadventure___parquet/joeySadventure--dataset_pokemon-8ffbcba98e672fcd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/liyuanhang/.cache/huggingface/datasets/joeySadventure___parquet/joeySadventure--dataset_pokemon-8ffbcba98e672fcd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cbae6713ef6c9558.arrow


(Dataset({
     features: ['pixel_values', 'input_ids'],
     num_rows: 833
 }),
 {'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]),
  'input_ids': tensor([49406,   320,  3610,   539,   320,  1901,  9528

In [4]:
#定义loader
def collate_fn(data):
    pixel_values = [i['pixel_values'] for i in data]
    input_ids = [i['input_ids'] for i in data]

    pixel_values = torch.stack(pixel_values).to(device)
    input_ids = torch.stack(input_ids).to(device)

    return {'pixel_values': pixel_values, 'input_ids': input_ids}


loader = torch.utils.data.DataLoader(dataset,
                                     shuffle=True,
                                     collate_fn=collate_fn,
                                     batch_size=1)

len(loader), next(iter(loader))

(833,
 {'pixel_values': tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]]]], device='cuda:0'),
  'input_ids': tensor([[49406,   320,  1674,   539,   320,  7651,  4009,   593,   320,  3801,
            

In [5]:
#加载模型
%run 1.encoder.ipynb
%run 2.vae.ipynb
%run 3.unet.ipynb

#准备训练
encoder.requires_grad_(False)
vae.requires_grad_(False)
unet.requires_grad_(True)

encoder.eval()
vae.eval()
unet.train()

encoder.to(device)
vae.to(device)
unet.to(device)

optimizer = torch.optim.AdamW(unet.parameters(),
                              lr=1e-5,
                              betas=(0.9, 0.999),
                              weight_decay=0.01,
                              eps=1e-8)

criterion = torch.nn.MSELoss()

optimizer, criterion

(AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 1e-05
     maximize: False
     weight_decay: 0.01
 ),
 MSELoss())

In [6]:
def get_loss(data):
    with torch.no_grad():
        #文字编码
        #[1, 77] -> [1, 77, 768]
        out_encoder = encoder(data['input_ids'])

        #抽取图像特征图
        #[1, 3, 512, 512] -> [1, 4, 64, 64]
        out_vae = vae.encoder(data['pixel_values'])
        out_vae = vae.sample(out_vae)

        #0.18215 = vae.config.scaling_factor
        out_vae = out_vae * 0.18215

    #随机数,unet的计算目标
    noise = torch.randn_like(out_vae)

    #往特征图中添加噪声
    #1000 = scheduler.num_train_timesteps
    #1 = batch size
    noise_step = torch.randint(0, 1000, (1, )).long().to(device)
    out_vae_noise = scheduler.add_noise(out_vae, noise, noise_step)

    #根据文字信息,把特征图中的噪声计算出来
    out_unet = unet(out_vae=out_vae_noise,
                    out_encoder=out_encoder,
                    time=noise_step)

    #计算mse loss
    #[1, 4, 64, 64],[1, 4, 64, 64]
    return criterion(out_unet, noise)


# get_loss({
#     'input_ids': torch.ones(1, 77, device=device).long(),
#     'pixel_values': torch.randn(1, 3, 512, 512, device=device)
# })

In [7]:
def train():
    loss_sum = 0
    for epoch in range(400):
        for i, data in enumerate(loader):
            loss = get_loss(data) / 4
            loss.backward()
            loss_sum += loss.item()

            if (epoch * len(loader) + i) % 4 == 0:
                torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()

        if epoch % 10 == 0:
            print(epoch, loss_sum)
            loss_sum = 0

    #torch.save(unet.to('cpu'), 'saves/unet.model')


train()

0 11.473097903071903
10 104.31830106899724


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│   18 │   #torch.save(unet.to('cpu'), 'saves/unet.model')                                         │
│   19                                                                                             │
│   20                                                                                             │
│ ❱ 21 train()                                                                                     │
│   22                                                                                             │
│                                                                                                  │
│ in train:6                                                                                       │
│                                                                                                  │
│    3 │   for epoch in range(400):                                                                │
│    4 │   │   for i, data in enumerate(loader):                                                   │
│    5 │   │   │   loss = get_loss(data) / 4                                                       │
│ ❱  6 │   │   │   loss.backward()                                                                 │
│    7 │   │   │   loss_sum += loss.item()                                                         │
│    8 │   │   │                                                                                   │
│    9 │   │   │   if (epoch * len(loader) + i) % 4 == 0:                                          │
│                                                                                                  │
│ /home/liyuanhang/miniconda3/envs/ldm/lib/python3.8/site-packages/torch/_tensor.py:363 in         │
│ backward                                                                                         │
│                                                                                                  │
│    360 │   │   │   │   retain_graph=retain_graph,                                                │
│    361 │   │   │   │   create_graph=create_graph,                                                │
│    362 │   │   │   │   inputs=inputs)                                                            │
│ ❱  363 │   │   torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=input  │
│    364 │                                                                                         │
│    365 │   def register_hook(self, hook):                                                        │
│    366 │   │   r"""Registers a backward hook.                                                    │
│                                                                                                  │
│ /home/liyuanhang/miniconda3/envs/ldm/lib/python3.8/site-packages/torch/autograd/__init__.py:173  │
│ in backward                                                                                      │
│                                                                                                  │
│   170 │   # The reason we repeat same the comment below is that                                  │
│   171 │   # some Python versions print out the first line of a multi-line function               │
│   172 │   # calls in the traceback and some print out the last line                              │
│ ❱ 173 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   174 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   175 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   176                                                      

In [1]:
from transformers import PreTrainedModel, PretrainedConfig


#包装类
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.unet = unet.to('cpu')

#保存到hub
Model(PretrainedConfig()).push_to_hub(
    repo_id='joeySadventure/sd_pokemon',
    use_auth_token=open("/home/liyuanhang/.cache/huggingface/token.txt").read().strip())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   │   self.unet = unet.to('cpu')                                                          │
│   11                                                                                             │
│   12 #保存到hub                                                                                  │
│ ❱ 13 Model(PretrainedConfig()).push_to_hub(                                                      │
│   14 │   repo_id='joeySadventure/sd_pokemon',                                                    │
│   15 │   use_auth_token=open("/home/liyuanhang/.cache/huggingface/token.txt").read().strip())    │
│   16                                                                                             │
│                                                                                                  │
│ in __init__:10                                                                                   │
│                                                                                                  │
│    7 │                                                                                           │
│    8 │   def __init__(self, config):                                                             │
│    9 │   │   super().__init__(config)                                                            │
│ ❱ 10 │   │   self.unet = unet.to('cpu')                                                          │
│   11                                                                                             │
│   12 #保存到hub                                                                                  │
│   13 Model(PretrainedConfig()).push_to_hub(                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'unet' is not defined